In [1]:
from collections import *
from heapq import *
from functools import *
from math import *
import numpy as np
import copy

data = None

with open('input.txt', 'r') as file:
    data = file.readlines()
    data = [d.strip() for d in data]
    data = [d.split(",") for d in data]
    data = [(int(x),int(y)) for x,y in data]
#print(data)

In [2]:
m,n = 70,70
byte = 1024
q = [(0,0)]
seen = set(q)
barriers = set(data[:byte])
def isEmpty(x,y):
    if 0<=x<=n and 0<=y <=m:
        return (x,y) not in barriers
    return False

v = 0
for step in range(1,m*n):
    v = step
    newq = [] 
    for x,y in q:
        for nx,ny in [[x,y+1],[x,y-1],[x-1,y],[x+1,y]]:
            tup = (nx,ny)
            if tup in seen:
                continue
            if not isEmpty(nx,ny):
                continue
            seen.add(tup)
            newq.append(tup)
    if (n,m) in seen or len(newq) == 0:
        break
    q = newq
print(v)


312


In [3]:
group = defaultdict(int)

def gen(x,y):
    return y*(m+1) + x
    
for y in range(m+1):
    for x in range(n+1):
        group[gen(x,y)] = gen(x,y)

def isEmpty(genxy):
    if genxy < 0 or genxy > gen(n,m):
        return False
    return (genxy) not in barriers

def find(genxy):
    if group[genxy] != genxy:
        group[genxy] = find(group[genxy])
    return group[genxy]

def join(agen,bgen):
    arep = find(agen)
    brep = find(bgen)
    group[arep] = brep
    return
    
def joinEdges(x,y):
    genxy = gen(x,y)
    if not isEmpty(genxy):
        return
    for nx,ny in [[x,y+1],[x,y-1],[x-1,y],[x+1,y]]:
        if 0 <= nx <=n and 0<= ny <=m:
            ngenxy = gen(nx,ny)
            if isEmpty(ngenxy):
                join(genxy,ngenxy)

barriers = set([gen(x,y) for x,y in data])
barrierq = data[:]

#initial 
for x in range(n+1):
    for y in range(m+1):
        joinEdges(x,y)
found = -1

while barrierq:
    bx,by = barrierq.pop()
    barriers.remove(gen(bx,by))
    joinEdges(bx,by)
    start = find(gen(0,0))
    end = find(gen(n,m))
    if start == end:
        found = (bx,by)
        break
print(found)

(28, 26)


In [4]:
import sys
import os

# Get the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
# Add the parent directory to sys.path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Now you can import the file
from viz import VideoGenerator

group = defaultdict(int)

def gen(x,y):
    return y*(m+1) + x
    
for y in range(m+1):
    for x in range(n+1):
        group[gen(x,y)] = gen(x,y)

def isEmpty(genxy):
    if genxy < 0 or genxy > gen(n,m):
        return False
    return (genxy) not in barriers

def find(genxy):
    if group[genxy] != genxy:
        group[genxy] = find(group[genxy])
    return group[genxy]

def join(agen,bgen):
    arep = find(agen)
    brep = find(bgen)
    group[arep] = brep
    return
    
def joinEdges(x,y):
    genxy = gen(x,y)
    if not isEmpty(genxy):
        return
    for nx,ny in [[x,y+1],[x,y-1],[x-1,y],[x+1,y]]:
        if 0 <= nx <=n and 0<= ny <=m:
            ngenxy = gen(nx,ny)
            if isEmpty(ngenxy):
                join(genxy,ngenxy)

barriers = set([gen(x,y) for x,y in data])
barrierq = data[:]

def getFrame(isFinished=False):
    start = find(gen(0,0))
    end = find(gen(n,m))
    startcolour = 2
    endcolour = 3
    if isFinished:
        startcolour = 4
        endcolour = 4
    frame = []
    for y in range(m+1):
        row = []
        for x in range(n+1):
            g = gen(x,y)
            group = find(g)
            if g in barriers:
                row.append(1)
                continue
            if group == start:
                row.append(startcolour)
                continue
            if group == end:
                row.append(endcolour)
                continue
            row.append(0)
            continue
        frame.append(row)
    return frame

colourMap = {
    0: (255,255,255), # white
    1: (100,100,100), # grey
    2: (255,0,0), #red
    3: (0,0,255), #blue
    4: (0,255,0), #green
}
fps = 24
Video = VideoGenerator("aoc2024day18",colourMap)

for i in range(fps*1): # hold for 1 secs
    Video.draw_frame(getFrame())
    Video.print_frame()


#initial 
for x in range(n+1):
    for y in range(m+1):
        joinEdges(x,y)
found = -1

for i in range(fps*1): # hold for 1 secs
    Video.draw_frame(getFrame())
    Video.print_frame()

while barrierq:
    bx,by = barrierq.pop()
    barriers.remove(gen(bx,by))
    Video.draw_frame(getFrame())
    Video.print_frame()
    joinEdges(bx,by)
    start = find(gen(0,0))
    end = find(gen(n,m))
    if start == end:
        found = (bx,by)
        break
    Video.draw_frame(getFrame())
    Video.print_frame()

for i in range(fps*3): # hold for 3 secs
    Video.draw_frame(getFrame(isFinished=True))
    Video.print_frame()
Video.print_video(framerate=fps)

(28, 26)


ffmpeg version 4.4 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 9.4.0 (GCC)
  configuration: --prefix=/home/josh/anaconda3 --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1634179802525/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/home/conda/feedstock_root/build_artifacts/ffmpeg_1634179802525/_build_env/bin/pkg-config
  libavutil      56. 70.100 / 56. 70.100
  libavcodec     58.134.100 / 58.134.100
  libavformat    58. 76.100 / 58. 76.100
  libavdevice    58. 13.100 / 58. 13.100
  libavfilter     7.110.100 /  7.110.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  9.100 /  5.  9.100
  libswresample   3.  9.100 /  3.  9.100
  libpostproc    55.  9.100 / 55.  9.